## MSNBC Wayback Scraper

In [1]:
import urllib.request
import gzip
import sys
import requests as rq
from bs4 import BeautifulSoup as bs
from time import sleep
from time import time
from random import randint
from IPython.core.display import clear_output
from warnings import warn
import json
import pandas as pd

In [19]:
# MSNBC archive urls
url = 'http://web.archive.org/cdx/search/cdx?url=nbcnews.com/politics&collapse=digest&from=20190401&to=20190431&output=json'

urls = rq.get(url).text
parse_url = json.loads(urls)
parse_url

[['urlkey',
  'timestamp',
  'original',
  'mimetype',
  'statuscode',
  'digest',
  'length'],
 ['com,nbcnews)/politics',
  '20190401012911',
  'https://www.nbcnews.com/politics',
  'text/html',
  '200',
  'FVZYAKIUIFOQY5NCP7AI4LJB4JNLYQOF',
  '38471'],
 ['com,nbcnews)/politics',
  '20190401014736',
  'https://www.nbcnews.com/politics',
  'text/html',
  '200',
  'HK6O3VELADSKC3TC6TVKXIZ4SFIWUQCK',
  '38557'],
 ['com,nbcnews)/politics',
  '20190401045810',
  'https://www.nbcnews.com/politics',
  'text/html',
  '200',
  'FKBB6H2CO2IDV7YDPUL4VEVXXE5OQFHN',
  '38549'],
 ['com,nbcnews)/politics',
  '20190401065204',
  'https://www.nbcnews.com/politics',
  'text/html',
  '200',
  'TGBH3GXDR7WFKAUUMPKWVT2E6TIPD7QK',
  '38473'],
 ['com,nbcnews)/politics',
  '20190401092652',
  'https://www.nbcnews.com/politics',
  'text/html',
  '200',
  'PUG672JFIZNX6ORIX5TOXXHGKXJTS5AP',
  '38554'],
 ['com,nbcnews)/politics',
  '20190401101307',
  'https://www.nbcnews.com/politics',
  'text/html',
  '200',


In [35]:
url_list = []
for i in range(1,len(parse_url)):
    orig_url = parse_url[i][2]
    tstamp = parse_url[i][1]
    waylink = tstamp+'/'+orig_url
    url_list.append(waylink)

In [36]:
url_list[:10]

['20190401012911/https://www.nbcnews.com/politics',
 '20190401014736/https://www.nbcnews.com/politics',
 '20190401045810/https://www.nbcnews.com/politics',
 '20190401065204/https://www.nbcnews.com/politics',
 '20190401092652/https://www.nbcnews.com/politics',
 '20190401101307/https://www.nbcnews.com/politics',
 '20190401122721/https://www.nbcnews.com/politics',
 '20190401123556/https://www.nbcnews.com/politics',
 '20190401144056/https://www.nbcnews.com/politics',
 '20190401144245/https://www.nbcnews.com/politics']

In [37]:
#open page
final_url = 'https://web.archive.org/web/'+url_list[0]

req = rq.get(final_url).text
# req = urllib.request.Request(final_url)
# with urllib.request.urlopen(req) as response:
#     page = response.read()

In [38]:
final_url

'https://web.archive.org/web/20190401012911/https://www.nbcnews.com/politics'

In [39]:
# parse html using beautifulsoup and store in soup
soup = bs(req,'html.parser')
soup

<!DOCTYPE html>
<html data-reactroot="" lang="en"><head><script src="//archive.org/includes/analytics.js?v=cf34f82" type="text/javascript"></script>
<script type="text/javascript">window.addEventListener('DOMContentLoaded',function(){var v=archive_analytics.values;v.service='wb';v.server_name='wwwb-app105.us.archive.org';v.server_ms=729;archive_analytics.send_pageview({});});</script><script charset="utf-8" src="/_static/js/ait-client-rewrite.js" type="text/javascript"></script>
<script type="text/javascript">
WB_wombat_Init("https://web.archive.org/web", "20190401012911", "www.nbcnews.com");
</script>
<script charset="utf-8" src="/_static/js/wbhack.js" type="text/javascript"></script>
<script type="text/javascript">
__wbhack.init('https://web.archive.org/web');
</script>
<link href="/_static/css/banner-styles.css" rel="stylesheet" type="text/css"/>
<link href="/_static/css/iconochive.css" rel="stylesheet" type="text/css"/>
<!-- End Wayback Rewrite JS Include -->
<title data-rh="true">

In [40]:
articles= soup.find_all('article')

In [86]:
articles[7].find_all('h2')[1].a.text
#title[1].a.text
#title[1].a['href']

'Kim showered Trump with flattery in letter before Hanoi summit'

In [88]:
## Creating a loop to scrape from all pages

news_title = []
#news_summary = []
news_source = []
#news_image = []
#news_caption = []
news_link = []

#pages = [str(i) for i in range(1,371)]

reqs = 0

start_time = time()

for url in url_list:
    
    full_url = 'https://web.archive.org/web/'+url
    
    #open page
    try:
        pg = rq.get(full_url).text
    except urllib.error.HTTPError as e:
        print('Error: {}'.format(e))
    except urllib.error.URLError as e:
        print('Error: {}'.format(e.reason))
        
    sleep(randint(10,20))
    reqs +=1
    
    # Calculate elapsed time between requests
    elapsed_time = time() - start_time
    print('Request: {}; Frequency: {} requests/s'.format(reqs,reqs/elapsed_time))
    

    
    #Break once the max pages is reached
    if reqs > len(url_list):
        warn('No. of requests was greater than expected')
        break
        
    # parse html using beautifulsoup and store in soup
    soup = bs(pg,'html.parser')
    
    #find all news containers
    articles = soup.find_all('article')
    
    # parse through news containers to get info
    for article in articles:
        try:
            
            if article != None:
                #title and link
                if article.find_all('h2') != None:
                    title = article.find_all('h2')[1].a.text 
                    link = article.find_all('h2')[1].a['href'] 
                else:
                    title = 'N/A'
                    link = 'N/A'
                # source
                source = 'NBC News'
            
#                 # Summary
#                 summ = article.find('div',attrs={'class':'excerpt'})
#                 if summ != None:
#                     summary = summ.p.text
#                 else:
#                     summary = 'N/A'
            
#                 #author
#                 author = article.footer.address['data-aname'] if article.footer.address else 'N/A'
            
#                 #image and caption
#                 if article.img != None:
#                     img = article.img['src']
#                     caption = article.img['alt']
        
            # Append data elements to lists
            news_title.append(title)
            #news_summary.append(summary)
            news_source.append(source)
            #news_image.append(img)
            #news_caption.append(caption)
            news_link.append(link)
        except:
            e = sys.exc_info()[0]
            print(e)

Request: 1; Frequency: 0.06335615142509061 requests/s
Request: 2; Frequency: 0.05609432316375105 requests/s
Request: 3; Frequency: 0.05811670088815762 requests/s
Request: 4; Frequency: 0.05969127219024806 requests/s
Request: 5; Frequency: 0.05977136865504576 requests/s
Request: 6; Frequency: 0.060711215367204086 requests/s
Request: 7; Frequency: 0.06316881357221654 requests/s
Request: 8; Frequency: 0.06154632285044971 requests/s
Request: 9; Frequency: 0.0618799605774569 requests/s
Request: 10; Frequency: 0.06331652221869492 requests/s
Request: 11; Frequency: 0.06262277107685055 requests/s
Request: 12; Frequency: 0.06230253447918314 requests/s
Request: 13; Frequency: 0.06322919264545808 requests/s
Request: 14; Frequency: 0.06167698549190843 requests/s
Request: 15; Frequency: 0.06182236286202546 requests/s
Request: 16; Frequency: 0.06070556994258397 requests/s
Request: 17; Frequency: 0.06099923175608263 requests/s
Request: 18; Frequency: 0.062023548923465656 requests/s
Request: 19; Frequ

Request: 149; Frequency: 0.06008458446922739 requests/s
Request: 150; Frequency: 0.05995827612427355 requests/s
Request: 151; Frequency: 0.05996442745525337 requests/s
Request: 152; Frequency: 0.05987902233496516 requests/s
Request: 153; Frequency: 0.059865641821644285 requests/s
Request: 154; Frequency: 0.05975052742970482 requests/s
Request: 155; Frequency: 0.0597817489895684 requests/s
Request: 156; Frequency: 0.059731982677665225 requests/s
Request: 157; Frequency: 0.059745644072508694 requests/s
Request: 158; Frequency: 0.05982569303014942 requests/s
Request: 159; Frequency: 0.0598443471545909 requests/s
Request: 160; Frequency: 0.059774071224427947 requests/s
Request: 161; Frequency: 0.05985390399359436 requests/s
Request: 162; Frequency: 0.05987703442976479 requests/s
Request: 163; Frequency: 0.05983054179516238 requests/s
Request: 164; Frequency: 0.05985749594397083 requests/s
Request: 165; Frequency: 0.05992875002882207 requests/s
Request: 166; Frequency: 0.05998943545469748 r

In [145]:
req = rq.get(news_link[0]).text
soup=bs(req,'html.parser')
article = soup.find('div',attrs={'class':'article container___2EGEI'})
article.h1.text
article.div.text
#image = article.find('div',attrs={'class':'heroMediaHolder___1X_U1 mb8'})
#article.article.text

"Most Americans don't think Trump is in the clear yet on Russia, new poll findsPresident Trump's approval remains stable and a third of voters say they don't know whether the summary of Mueller's findings clears him of wrongdoing in a new NBC News/WSJ poll."

In [143]:
news_link[5]

'https://web.archive.org/web/20190401012911/https://www.nbcnews.com/politics/white-house/trump-takes-victory-lap-russia-hoax-finally-dead-n988696'

In [ ]:
## Creating a loop to scrape summary from links

#news_title = []
news_summary = []
#news_source = []
#news_image = []
#news_caption = []
summ_link = []

#pages = [str(i) for i in range(1,371)]

reqs = 0

start_time = time()

for url in news_link:
    
    #full_url = 'https://web.archive.org/web/'+url
    
    #open page
    try:
        pg = rq.get(url).text
    except urllib.error.HTTPError as e:
        print('Error: {}'.format(e))
    except urllib.error.URLError as e:
        print('Error: {}'.format(e.reason))
        
    sleep(randint(10,20))
    reqs +=1
    
    # Calculate elapsed time between requests
    elapsed_time = time() - start_time
    print('Request: {}; Frequency: {} requests/s'.format(reqs,reqs/elapsed_time))
    

    
    #Break once the max pages is reached
    if reqs > len(news_link):
        warn('No. of requests was greater than expected')
        break
        
    # parse html using beautifulsoup and store in soup
    soup = bs(pg,'html.parser')
    
    #find all news containers
    article = soup.find('div',attrs={'class':'article container___2EGEI'})
    try:
        if article.div != None:
            summ = article.div.text
        else:
            summ = 'N/A'
        
        news_summary.append(summ)
        summ_link.append(url)
    except:
        e = sys.exc_info()[0]
        print(e)


Request: 1; Frequency: 0.06840193750998169 requests/s
Request: 2; Frequency: 0.0639927210428007 requests/s
Request: 3; Frequency: 0.060158839667041754 requests/s
Request: 4; Frequency: 0.06135501499075311 requests/s
Request: 5; Frequency: 0.06358050410743642 requests/s
Request: 6; Frequency: 0.06383970262656831 requests/s
Request: 7; Frequency: 0.06220993103991405 requests/s
Request: 8; Frequency: 0.062139676256380755 requests/s
Request: 9; Frequency: 0.06363199215218485 requests/s
Request: 10; Frequency: 0.0615151396016292 requests/s
<class 'AttributeError'>
Request: 11; Frequency: 0.060260742895774794 requests/s
Request: 12; Frequency: 0.058706882934093395 requests/s
Request: 13; Frequency: 0.058893914710428596 requests/s
Request: 14; Frequency: 0.058826308362848086 requests/s
Request: 15; Frequency: 0.058709355355711965 requests/s
Request: 16; Frequency: 0.06008866360886628 requests/s
Request: 17; Frequency: 0.060220616797270726 requests/s
Request: 18; Frequency: 0.06039754457376442

Request: 145; Frequency: 0.06022839741901516 requests/s
Request: 146; Frequency: 0.06022728968897803 requests/s
Request: 147; Frequency: 0.060249950196752854 requests/s
Request: 148; Frequency: 0.06021717494957206 requests/s
Request: 149; Frequency: 0.06027933252622759 requests/s
Request: 150; Frequency: 0.06042405566724251 requests/s
Request: 151; Frequency: 0.06040981783980287 requests/s
Request: 152; Frequency: 0.06032152089644622 requests/s
Request: 153; Frequency: 0.06026882333808635 requests/s
Request: 154; Frequency: 0.06024296180706298 requests/s
Request: 155; Frequency: 0.060194211686407165 requests/s
Request: 156; Frequency: 0.06021670538229059 requests/s
Request: 157; Frequency: 0.060232286834960416 requests/s
<class 'AttributeError'>
Request: 158; Frequency: 0.06034437377649261 requests/s
Request: 159; Frequency: 0.0603775681938631 requests/s
Request: 160; Frequency: 0.06025782328586842 requests/s
Request: 161; Frequency: 0.06036836656014138 requests/s
Request: 162; Frequen

Request: 287; Frequency: 0.06057943071136116 requests/s
Request: 288; Frequency: 0.06052405888033995 requests/s
Request: 289; Frequency: 0.060576353741515716 requests/s
Request: 290; Frequency: 0.06052256933786469 requests/s
Request: 291; Frequency: 0.06048953772822006 requests/s
<class 'AttributeError'>
Request: 292; Frequency: 0.06048388676881978 requests/s
Request: 293; Frequency: 0.06044307725069262 requests/s
Request: 294; Frequency: 0.06045705922312422 requests/s
Request: 295; Frequency: 0.060527681401180686 requests/s
Request: 296; Frequency: 0.06054881813701633 requests/s
Request: 297; Frequency: 0.060504176249204594 requests/s
Request: 298; Frequency: 0.06048712210654221 requests/s
Request: 299; Frequency: 0.060535978561847 requests/s
Request: 300; Frequency: 0.06048468428785532 requests/s
Request: 301; Frequency: 0.06045902015438739 requests/s
Request: 302; Frequency: 0.0604151051677264 requests/s
Request: 303; Frequency: 0.06040754929843686 requests/s
Request: 304; Frequency

Request: 426; Frequency: 0.06097895457112505 requests/s
<class 'AttributeError'>
Request: 427; Frequency: 0.06098438002833828 requests/s
Request: 428; Frequency: 0.06096090524376747 requests/s
Request: 429; Frequency: 0.06094935905192065 requests/s
Request: 430; Frequency: 0.060979972676284085 requests/s
Request: 431; Frequency: 0.060981771578218075 requests/s
Request: 432; Frequency: 0.061004712825094974 requests/s
Request: 433; Frequency: 0.061034539980781254 requests/s
Request: 434; Frequency: 0.061039623435813845 requests/s
Request: 435; Frequency: 0.0610430512358318 requests/s
Request: 436; Frequency: 0.06108945291699517 requests/s
Request: 437; Frequency: 0.06110721711702236 requests/s
<class 'AttributeError'>
Request: 438; Frequency: 0.061084374172140056 requests/s
<class 'AttributeError'>
Request: 439; Frequency: 0.061058610485118466 requests/s
Request: 440; Frequency: 0.06109684613835689 requests/s
Request: 441; Frequency: 0.06110891601316707 requests/s
Request: 442; Frequency

Request: 568; Frequency: 0.061869057786943384 requests/s
Request: 569; Frequency: 0.061837636869286186 requests/s
Request: 570; Frequency: 0.06180642005381575 requests/s
Request: 571; Frequency: 0.06178368563009743 requests/s
Request: 572; Frequency: 0.061769718325686125 requests/s
Request: 573; Frequency: 0.061776924955409096 requests/s
Request: 574; Frequency: 0.061803974100065 requests/s
Request: 575; Frequency: 0.06177907428131508 requests/s
Request: 576; Frequency: 0.061772084342931896 requests/s
Request: 577; Frequency: 0.06176118056211755 requests/s
Request: 578; Frequency: 0.061738384362182444 requests/s
Request: 579; Frequency: 0.06173222197113889 requests/s
Request: 580; Frequency: 0.06173345085148748 requests/s
Request: 581; Frequency: 0.06172283408155368 requests/s
Request: 582; Frequency: 0.06170566238634212 requests/s
Request: 583; Frequency: 0.06169216899561239 requests/s
Request: 584; Frequency: 0.061707600524100265 requests/s
Request: 585; Frequency: 0.0617369456356374

...

Request: 1116; Frequency: 0.06124152040401188 requests/s
Request: 1117; Frequency: 0.06125636195006008 requests/s
Request: 1118; Frequency: 0.0612448320987235 requests/s
<class 'AttributeError'>
Request: 1119; Frequency: 0.06125294679548974 requests/s
Request: 1120; Frequency: 0.06125789638730079 requests/s
Request: 1121; Frequency: 0.06125289443849866 requests/s
Request: 1122; Frequency: 0.061264542991033434 requests/s
Request: 1123; Frequency: 0.06127656075843529 requests/s
Request: 1124; Frequency: 0.06126837500378287 requests/s
Request: 1125; Frequency: 0.06127636607687442 requests/s
Request: 1126; Frequency: 0.06128009057029955 requests/s
Request: 1127; Frequency: 0.0612916182712043 requests/s
Request: 1128; Frequency: 0.0612829761113872 requests/s
Request: 1129; Frequency: 0.06126758741134902 requests/s
Request: 1130; Frequency: 0.06128184219423691 requests/s
<class 'AttributeError'>
Request: 1131; Frequency: 0.061266771616151974 requests/s
Request: 1132; Frequency: 0.06127804911

Request: 1258; Frequency: 0.06135077688368585 requests/s
Request: 1259; Frequency: 0.06134925883568776 requests/s
Request: 1260; Frequency: 0.06135156467312337 requests/s
<class 'AttributeError'>
Request: 1261; Frequency: 0.0613663785641518 requests/s
Request: 1262; Frequency: 0.061361579210133556 requests/s
Request: 1263; Frequency: 0.06136836404517751 requests/s
Request: 1264; Frequency: 0.061371414959217906 requests/s
Request: 1265; Frequency: 0.06136931101487386 requests/s
Request: 1266; Frequency: 0.06138557008090224 requests/s
Request: 1267; Frequency: 0.06137482399437717 requests/s
Request: 1268; Frequency: 0.06139136109805293 requests/s
Request: 1269; Frequency: 0.06139142314990008 requests/s
Request: 1270; Frequency: 0.06138627733335862 requests/s
Request: 1271; Frequency: 0.06139623976364128 requests/s
Request: 1272; Frequency: 0.061408955297650364 requests/s
Request: 1273; Frequency: 0.061401099703492924 requests/s
Request: 1274; Frequency: 0.061407760697104075 requests/s
Re

Request: 1397; Frequency: 0.061345752124324165 requests/s
Request: 1398; Frequency: 0.06135345906584991 requests/s
Request: 1399; Frequency: 0.061340470534330366 requests/s
Request: 1400; Frequency: 0.0613354939802238 requests/s
<class 'AttributeError'>
Request: 1401; Frequency: 0.06132596605006505 requests/s
Request: 1402; Frequency: 0.06131832670171179 requests/s
Request: 1403; Frequency: 0.06131502850955785 requests/s
Request: 1404; Frequency: 0.061326838949774885 requests/s
Request: 1405; Frequency: 0.061338325382811164 requests/s
Request: 1406; Frequency: 0.06134854588062789 requests/s
Request: 1407; Frequency: 0.06132978204169114 requests/s
Request: 1408; Frequency: 0.061327082065657866 requests/s
Request: 1409; Frequency: 0.061318938563536436 requests/s
Request: 1410; Frequency: 0.06132230518002002 requests/s
Request: 1411; Frequency: 0.061335050446752505 requests/s
Request: 1412; Frequency: 0.06132022630109965 requests/s
Request: 1413; Frequency: 0.0613276233544664 requests/s
R

Request: 1533; Frequency: 0.061190933136209685 requests/s
Request: 1534; Frequency: 0.061201621913009895 requests/s
Request: 1535; Frequency: 0.06119238752927896 requests/s
Request: 1536; Frequency: 0.05951010550863683 requests/s
Request: 1537; Frequency: 0.0595041622051008 requests/s
Request: 1538; Frequency: 0.05949631855675982 requests/s
Request: 1539; Frequency: 0.05949706063340814 requests/s
Request: 1540; Frequency: 0.059509479133947704 requests/s
Request: 1541; Frequency: 0.05950669811518536 requests/s
Request: 1542; Frequency: 0.05951984657318892 requests/s
Request: 1543; Frequency: 0.059527765992261825 requests/s
Request: 1544; Frequency: 0.0595233659054771 requests/s
<class 'AttributeError'>
Request: 1545; Frequency: 0.059522682727180136 requests/s
Request: 1546; Frequency: 0.059524766244961475 requests/s
Request: 1547; Frequency: 0.059533065301834394 requests/s
Request: 1548; Frequency: 0.05953001907523669 requests/s
Request: 1549; Frequency: 0.05952139791682486 requests/s
R

Request: 1673; Frequency: 0.05967676674981392 requests/s
Request: 1674; Frequency: 0.05967004598144118 requests/s
Request: 1675; Frequency: 0.059661372574114586 requests/s
Request: 1676; Frequency: 0.0596697988596359 requests/s
Request: 1677; Frequency: 0.05968045065046456 requests/s
Request: 1678; Frequency: 0.05969346712271391 requests/s
Request: 1679; Frequency: 0.0596953595825469 requests/s
Request: 1680; Frequency: 0.059703826732019724 requests/s
Request: 1681; Frequency: 0.059695312546345025 requests/s
<class 'AttributeError'>
Request: 1682; Frequency: 0.05969667250086394 requests/s
Request: 1683; Frequency: 0.05969194924337224 requests/s
Request: 1684; Frequency: 0.059685179640234814 requests/s
Request: 1685; Frequency: 0.05968348056021344 requests/s
Request: 1686; Frequency: 0.05968842916707617 requests/s
Request: 1687; Frequency: 0.05970000790081581 requests/s
Request: 1688; Frequency: 0.05971019390022587 requests/s
Request: 1689; Frequency: 0.0597164464199836 requests/s
Reque

In [153]:
summ_link

['https://web.archive.org/web/20190401012911/https://www.nbcnews.com/politics/meet-the-press/poll-after-mueller-summary-americans-are-still-wait-see-mode-n989061',
 'https://web.archive.org/web/20190401012911/https://www.nbcnews.com/politics/politics-news/beto-o-rourke-kicks-three-texas-rallies-focus-border-n989206',
 'https://web.archive.org/web/20190401012911/https://www.nbcnews.com/politics/2020-election/massive-student-debt-piling-so-are-candidates-solutions-n987741']

In [1]:
import pandas as pd

nbc_df = pd.DataFrame({'title':news_title
                            #,'summary':news_summary
                            ,'source':news_source
                            #,'image':news_image
                            #,'img_caption':news_caption
                            ,'article_link':news_link})
nbc_df.to_csv('nbc_articles.csv',index=False)

KeyboardInterrupt: 

## NYTimes Wayback Scraper

In [82]:
# NYTimes archive urls
url = 'http://web.archive.org/cdx/search/cdx?url=nytimes.com/section/politics&collapse=timestamp:10&from=20190401&to=20190431&output=json'

urls = rq.get(url).text
parse_url = json.loads(urls)
parse_url

[['urlkey',
  'timestamp',
  'original',
  'mimetype',
  'statuscode',
  'digest',
  'length'],
 ['com,nytimes)/section/politics',
  '20190401015034',
  'https://www.nytimes.com/section/politics',
  'text/html',
  '200',
  '6NQBZ72WTJDE32EZMIWKNZJ2PSFH6AGI',
  '75088'],
 ['com,nytimes)/section/politics',
  '20190401022815',
  'https://www.nytimes.com/section/politics',
  'text/html',
  '200',
  'EXCOJC2EZES2CHDQOUJQ5WKXZET5O5BF',
  '74932'],
 ['com,nytimes)/section/politics',
  '20190401091708',
  'https://www.nytimes.com/section/politics',
  'text/html',
  '200',
  'BJDCZEN5U3464APGS65G5V65HESG3M45',
  '79509'],
 ['com,nytimes)/section/politics',
  '20190401130023',
  'https://www.nytimes.com/section/politics',
  'text/html',
  '200',
  'FH7UTMM32X6MUBPVMX6O4PE3MZJJXNFP',
  '79716'],
 ['com,nytimes)/section/politics',
  '20190401143215',
  'https://www.nytimes.com/section/politics',
  'text/html',
  '200',
  'WVQ755XQKAFWPYWUB3ZMOI44V5JYOCRC',
  '74411'],
 ['com,nytimes)/section/polit

In [83]:
url_list = []
for i in range(1,len(parse_url)):
    orig_url = parse_url[i][2]
    tstamp = parse_url[i][1]
    waylink = tstamp+'/'+orig_url
    url_list.append(waylink)
url_list[:10]

['20190401015034/https://www.nytimes.com/section/politics',
 '20190401022815/https://www.nytimes.com/section/politics',
 '20190401091708/https://www.nytimes.com/section/politics',
 '20190401130023/https://www.nytimes.com/section/politics',
 '20190401143215/https://www.nytimes.com/section/politics',
 '20190401163023/https://www.nytimes.com/section/politics',
 '20190401212126/https://www.nytimes.com/section/politics',
 '20190401221434/https://www.nytimes.com/section/politics',
 '20190402015109/https://www.nytimes.com/section/politics',
 '20190402023147/https://www.nytimes.com/section/politics']

In [84]:
len(url_list)

208

In [5]:
#open page
final_url = 'https://web.archive.org/web/'+url_list[0]

req = rq.get(final_url).text

In [6]:
soup = bs(req,'html.parser')
soup

<!DOCTYPE html>

<html lang="en" xmlns:og="http://opengraphprotocol.org/schema/">
<head><script src="//archive.org/includes/analytics.js?v=cf34f82" type="text/javascript"></script>
<script type="text/javascript">window.addEventListener('DOMContentLoaded',function(){var v=archive_analytics.values;v.service='wb';v.server_name='wwwb-app22.us.archive.org';v.server_ms=2257;archive_analytics.send_pageview({});});</script><script charset="utf-8" src="/_static/js/ait-client-rewrite.js" type="text/javascript"></script>
<script type="text/javascript">
WB_wombat_Init("https://web.archive.org/web", "20190401015034", "www.nytimes.com");
</script>
<script charset="utf-8" src="/_static/js/wbhack.js" type="text/javascript"></script>
<script type="text/javascript">
__wbhack.init('https://web.archive.org/web');
</script>
<link href="/_static/css/banner-styles.css" rel="stylesheet" type="text/css"/>
<link href="/_static/css/iconochive.css" rel="stylesheet" type="text/css"/>
<!-- End Wayback Rewrite JS In

In [17]:
articles1= soup.find_all('li',attrs={'class':'css-12ro1rs ekkqrpp3'})
len(articles1)

2

In [18]:
articles2=soup.find_all('li',attrs={'class':'css-1nzomwi'})
len(articles2)

2

In [20]:
articles3 = soup.find_all('li',attrs={'class':'css-ye6x8s'})
len(articles3)

10

In [33]:
articles = articles1+articles2+articles3
#title
articles[13].h2.text

'Matt Gaetz Is a Congressman Liberals Love to Loathe. It’s All Part of the Plan.'

In [34]:
#summary
articles[13].p.text

'Mr. Gaetz came to Washington as the son of the Florida Senate president, with a reputation for deal making. His incendiary antics have put that rap behind him.'

In [38]:
#link
articles[3].a['href']

'/web/20190401015034/https://www.nytimes.com/2019/03/31/us/politics/elizabeth-warren-fundraising.html'

In [48]:
#author
#articles[1].find('p',attrs={'class':'css-1xonkmu'}).text
articles[3].find('p',attrs={'class':'css-1lwiyax e4e4i5l2'}).text

'By Astead W. Herndon and Jonathan Martin'

In [87]:
## Creating a loop to scrape from all pages

news_title = []
news_summary = []
news_source = []
news_author = []
#news_caption = []
news_link = []

#pages = [str(i) for i in range(1,371)]

reqs = 0

start_time = time()

for url in url_list:
    
    full_url = 'https://web.archive.org/web/'+url
    
    #open page
    try:
        pg = rq.get(full_url).text
    except urllib.error.HTTPError as e:
        print('Error: {}'.format(e))
    except urllib.error.URLError as e:
        print('Error: {}'.format(e.reason))
        
    sleep(randint(10,20))
    reqs +=1
    
    # Calculate elapsed time between requests
    elapsed_time = time() - start_time
    print('Request: {}; Frequency: {} requests/s'.format(reqs,reqs/elapsed_time))
    

    
    #Break once the max pages is reached
    if reqs > len(url_list):
        warn('No. of requests was greater than expected')
        break
        
    # parse html using beautifulsoup and store in soup
    soup = bs(pg,'html.parser')
    
    #find all news containers
    articles1= soup.find_all('li',attrs={'class':'css-12ro1rs ekkqrpp3'})
    articles2=soup.find_all('li',attrs={'class':'css-1nzomwi'})
    articles3 = soup.find_all('li',attrs={'class':'css-ye6x8s'})
    
    articles = articles1+articles2+articles3
    
    # parse through news containers to get info
    for article in articles:
        try:
            
            if article != None:
                #title
                if article.h2 != None:
                    title = article.h2.text  
                else:
                    title = 'N/A'
                #summary
                if article.p != None:
                    summ = article.p.text
                else:
                    summ = 'N/A'
                #link
                if article.a != None:
                    link = article.a['href']
                else:
                    link = 'N/A'
                
                if article.find('p',attrs={'class':'css-1lwiyax e4e4i5l2'}) != None:
                    author = article.find('p',attrs={'class':'css-1lwiyax e4e4i5l2'}).text
                elif article.find('p',attrs={'class':'css-1xonkmu'}) != None:
                    author = article.find('p',attrs={'class':'css-1xonkmu'}).text
                else:
                    author = 'N/A'
                
                # source
                source = 'NYTimes Politics'
            
#                 # Summary
#                 summ = article.find('div',attrs={'class':'excerpt'})
#                 if summ != None:
#                     summary = summ.p.text
#                 else:
#                     summary = 'N/A'
            
#                 #author
#                 author = article.footer.address['data-aname'] if article.footer.address else 'N/A'
            
#                 #image and caption
#                 if article.img != None:
#                     img = article.img['src']
#                     caption = article.img['alt']
        
            # Append data elements to lists
            news_title.append(title)
            news_summary.append(summ)
            news_source.append(source)
            news_author.append(author)
            #news_caption.append(caption)
            news_link.append(link)
        except:
            e = sys.exc_info()[0]
            print(e)

Request: 1; Frequency: 0.049860590075263944 requests/s
Request: 2; Frequency: 0.05551900040109372 requests/s
Request: 3; Frequency: 0.05235761371576515 requests/s
Request: 4; Frequency: 0.05176802532136932 requests/s
Request: 5; Frequency: 0.05013469860933653 requests/s
Request: 6; Frequency: 0.04939534956627347 requests/s
Request: 7; Frequency: 0.0498494810311327 requests/s
Request: 8; Frequency: 0.05047983797531596 requests/s
Request: 9; Frequency: 0.05095660012659258 requests/s
Request: 10; Frequency: 0.05156000749318884 requests/s
Request: 11; Frequency: 0.05163055177325009 requests/s
Request: 12; Frequency: 0.05216610183569642 requests/s
Request: 13; Frequency: 0.05212782182607244 requests/s
Request: 14; Frequency: 0.052532000663120575 requests/s
Request: 15; Frequency: 0.0537542971559576 requests/s
Request: 16; Frequency: 0.05489521337423884 requests/s
Request: 17; Frequency: 0.055670397824263286 requests/s
Request: 18; Frequency: 0.055874345115391265 requests/s
Request: 19; Freq

Request: 149; Frequency: 0.0564481482545101 requests/s
Request: 150; Frequency: 0.05650550749696125 requests/s
Request: 151; Frequency: 0.056610330877016446 requests/s
Request: 152; Frequency: 0.05666689903174352 requests/s
Request: 153; Frequency: 0.05662131586387303 requests/s
Request: 154; Frequency: 0.056731779958559624 requests/s
Request: 155; Frequency: 0.056799286526299106 requests/s
Request: 156; Frequency: 0.05678217141400077 requests/s
Request: 157; Frequency: 0.056799892975337996 requests/s
Request: 158; Frequency: 0.05687298364406978 requests/s
Request: 159; Frequency: 0.05680533772113289 requests/s
Request: 160; Frequency: 0.05677723417129974 requests/s
Request: 161; Frequency: 0.0568693164133493 requests/s
Request: 162; Frequency: 0.056883012443120044 requests/s
Request: 163; Frequency: 0.05683441381237085 requests/s
Request: 164; Frequency: 0.05681039319490636 requests/s
Request: 165; Frequency: 0.05674649304156843 requests/s
Request: 166; Frequency: 0.056822033447248614

In [1]:
import pandas as pd

nytimes_df = pd.DataFrame({'title':news_title
                            ,'summary':news_summary
                            ,'source':news_source
                            ,'author':news_author
                            #,'img_caption':news_caption
                            ,'article_link':news_link})
nytimes_df.to_csv('nytimes1_articles.csv',index=False)

NameError: name 'news_title' is not defined

In [99]:
import csv

rows = zip(news_title,news_summary,news_author,news_link)

with open('NYTimes_articles.csv', "w") as f:
    writer = csv.writer(f)
    for row in rows:
        writer.writerow(row)



## FOXNEWS Wayback Scraper

In [46]:
# Foxnews politics archive urls
url = 'http://web.archive.org/cdx/search/cdx?url=foxnews.com/politics&collapse=timestamp:10&from=20190301&to=20190431&filter=!statuscode:404&output=json'

urls = rq.get(url).text
parse_url = json.loads(urls)
parse_url

[['urlkey',
  'timestamp',
  'original',
  'mimetype',
  'statuscode',
  'digest',
  'length'],
 ['com,foxnews)/politics',
  '20190301053130',
  'http://www.foxnews.com/politics/',
  'unk',
  '301',
  '3I42H3S6NNFQ2MSVX7XZKYAYSCX5QBYJ',
  '416'],
 ['com,foxnews)/politics',
  '20190301092529',
  'http://foxnews.com/politics',
  'unk',
  '301',
  '3I42H3S6NNFQ2MSVX7XZKYAYSCX5QBYJ',
  '367'],
 ['com,foxnews)/politics',
  '20190301132703',
  'https://www.foxnews.com/politics',
  'text/html',
  '200',
  'FDFVXOUKMWROJBHY333SGH6X5TY7E34Q',
  '21977'],
 ['com,foxnews)/politics',
  '20190302010839',
  'https://www.foxnews.com/politics',
  'text/html',
  '200',
  'GYVI3DY2D3VYPDSOQK4AUSLO424QNYDZ',
  '20882'],
 ['com,foxnews)/politics',
  '20190302024524',
  'http://foxnews.com/politics',
  'unk',
  '301',
  '3I42H3S6NNFQ2MSVX7XZKYAYSCX5QBYJ',
  '369'],
 ['com,foxnews)/politics',
  '20190302095144',
  'https://www.foxnews.com/politics',
  'text/html',
  '200',
  '2K4B4UG2ZD5BJ2EAPSRHOQKIEX6CUMP

In [47]:
url_list = []
for i in range(1,len(parse_url)):
    orig_url = parse_url[i][2]
    tstamp = parse_url[i][1]
    waylink = tstamp+'/'+orig_url
    url_list.append(waylink)
url_list[:10]

['20190301053130/http://www.foxnews.com/politics/',
 '20190301092529/http://foxnews.com/politics',
 '20190301132703/https://www.foxnews.com/politics',
 '20190302010839/https://www.foxnews.com/politics',
 '20190302024524/http://foxnews.com/politics',
 '20190302095144/https://www.foxnews.com/politics',
 '20190302122955/https://www.foxnews.com/politics',
 '20190302185038/https://www.foxnews.com/politics/',
 '20190303015954/https://www.foxnews.com/politics',
 '20190303024743/http://foxnews.com/politics']

In [48]:
len(url_list)

157

In [49]:
#open page
final_url = 'https://web.archive.org/web/'+url_list[0]

req = rq.get(final_url).text

In [36]:
final_url

'https://web.archive.org/web/20190401042743/http://foxnews.com/politics'

In [50]:
soup = bs(req,'html.parser')
soup

<!DOCTYPE html>

<html data-n-head="" data-n-head-ssr="">
<head><script src="//archive.org/includes/analytics.js?v=cf34f82" type="text/javascript"></script>
<script type="text/javascript">window.addEventListener('DOMContentLoaded',function(){var v=archive_analytics.values;v.service='wb';v.server_name='wwwb-app100.us.archive.org';v.server_ms=514;archive_analytics.send_pageview({});});</script><script charset="utf-8" src="/_static/js/ait-client-rewrite.js" type="text/javascript"></script>
<script type="text/javascript">
WB_wombat_Init("https://web.archive.org/web", "20190301092529", "www.foxnews.com");
</script>
<script charset="utf-8" src="/_static/js/wbhack.js" type="text/javascript"></script>
<script type="text/javascript">
__wbhack.init('https://web.archive.org/web');
</script>
<link href="/_static/css/banner-styles.css" rel="stylesheet" type="text/css"/>
<link href="/_static/css/iconochive.css" rel="stylesheet" type="text/css"/>
<!-- End Wayback Rewrite JS Include -->
<meta content=

In [51]:
articles = soup.find_all('article')
len(articles)

40

In [52]:
title = articles[0].find(attrs={'class':'title'}).text
title

'How House Republicans are making life difficult for Democrats, Pelosi'

In [53]:
summary = articles[0].p.text
summary

'House Republicans are driving Democrats crazy, and driving a major wedge through the majority party.'

In [54]:
link = articles[0].find(attrs={'class':'title'}).a['href']
link

'/web/20190301092529/https://www.foxnews.com/politics/how-house-republicans-are-making-life-difficult-for-democrats-pelosi'

In [55]:
fox_topic = articles[0].find(attrs={'class':'eyebrow'}).a.text
fox_topic

'Capitol Attitude'

In [56]:
## Creating a loop to scrape from all pages

news_title = []
news_summary = []
news_source = []
news_topic = []
#news_caption = []
news_link = []

#pages = [str(i) for i in range(1,371)]

reqs = 0

start_time = time()

for url in url_list:
    
    full_url = 'https://web.archive.org/web/'+url
    
    #open page
    try:
        pg = rq.get(full_url).text
    except urllib.error.HTTPError as e:
        print('Error: {}'.format(e))
    except urllib.error.URLError as e:
        print('Error: {}'.format(e.reason))
        
    sleep(randint(10,20))
    reqs +=1
    
    # Calculate elapsed time between requests
    elapsed_time = time() - start_time
    print('Request: {}; Frequency: {} requests/s'.format(reqs,reqs/elapsed_time))
    

    
    #Break once the max pages is reached
    if reqs > len(url_list):
        warn('No. of requests was greater than expected')
        break
        
    # parse html using beautifulsoup and store in soup
    soup = bs(pg,'html.parser')
    
    #find all news containers
    
    articles = soup.find_all('article')
    
    # parse through news containers to get info
    for article in articles:
        try:
            
            if article != None:
                #title, link
                if article.find(attrs={'class':'title'}) != None:
                    title = article.find(attrs={'class':'title'}).text
                    link = article.find(attrs={'class':'title'}).a['href']
                else:
                    title = 'N/A'
                    link = 'N/A'
                #summary
                if article.p != None:
                    summ = article.p.text
                else:
                    summ = 'N/A'
                
                # Fox topic
                if article.find(attrs={'class':'eyebrow'}) != None:
                    fox_topic = article.find(attrs={'class':'eyebrow'}).a.text
                else:
                    fox_topic = 'N/A'
                
                # source
                source = 'FoxNews Politics'

        
            # Append data elements to lists
            news_title.append(title)
            news_summary.append(summ)
            news_source.append(source)
            news_topic.append(fox_topic)
            #news_caption.append(caption)
            news_link.append(link)
        except:
            e = sys.exc_info()[0]
            print(e)

Request: 1; Frequency: 0.09347450392459718 requests/s
Request: 2; Frequency: 0.06506081057986457 requests/s
Request: 3; Frequency: 0.063131180757911 requests/s
Request: 4; Frequency: 0.06347299045082964 requests/s
Request: 5; Frequency: 0.06200915710406137 requests/s
Request: 6; Frequency: 0.06120408678543379 requests/s
Request: 7; Frequency: 0.05941299993428578 requests/s
Request: 8; Frequency: 0.06060055799330466 requests/s
Request: 9; Frequency: 0.06271462400176801 requests/s
Request: 10; Frequency: 0.06381272286183864 requests/s
Request: 11; Frequency: 0.06334628746580528 requests/s
Request: 12; Frequency: 0.06207879504467736 requests/s
Request: 13; Frequency: 0.06220726509461113 requests/s
Request: 14; Frequency: 0.062130977382651595 requests/s
Request: 15; Frequency: 0.06257915276272137 requests/s
Request: 16; Frequency: 0.06239246478756193 requests/s
Request: 17; Frequency: 0.061143936094888704 requests/s
Request: 18; Frequency: 0.06063392262564888 requests/s
Request: 19; Freque

Request: 149; Frequency: 0.05855755851926302 requests/s
Request: 150; Frequency: 0.058537659591608705 requests/s
Request: 151; Frequency: 0.05842186696139113 requests/s
Request: 152; Frequency: 0.058323538570043895 requests/s
Request: 153; Frequency: 0.058440700220723465 requests/s
Request: 154; Frequency: 0.05841819277974578 requests/s
Request: 155; Frequency: 0.05840493003153793 requests/s
Request: 156; Frequency: 0.05848773593472577 requests/s
Request: 157; Frequency: 0.05842508658434235 requests/s


In [57]:
fox_df = pd.DataFrame({'title':news_title,
                      'summary':news_summary
                      ,'topic':news_topic
                      ,'link':news_link})
fox_df.to_csv('FOXNews_articles.csv',index=False)